In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from transformers import pipeline
from pygaur.gnlp import get_df_tags_keywords, load_extracted_file
import pandas as pd

import re

## Import data

In [16]:
action_tag_path = "/home/gquetel/repos/gaur/pygaur/pygaur/data/mysql/tags/actions.tags"
df_actions = get_df_tags_keywords(action_tag_path)
labels_actions = list(set(df_actions['tag'].values))


In [17]:
extracted_filepath = (
    "/home/gquetel/repos/gaur/pygaur/pygaur/data/mysql/extracted/mysql.gaur"
)
df_extracted = load_extracted_file(extracted_filepath)


def construct_documents(df_ex: pd.DataFrame) -> pd.DataFrame:
    index = df_ex.index.values
    documents = (
        # Remove the suffix part of the rule identifier.
        df_ex.index.to_series().apply(lambda x: re.sub(r"\.\d+$", "", str(x)))
        + " "
        + df_ex["terminals"].astype(str)
        + df_ex["code"].astype(str)
    )
    df_docs = pd.DataFrame(documents, columns=["doc"], index=index)

    df_docs["doc"] = df_docs["doc"].str.replace("_", " ")
    df_docs["doc"] = df_docs["doc"].str.lower()

    return df_docs

df_docs = construct_documents(df_extracted)


In [18]:
df_docs = df_docs.head(100)      

In [19]:
pipe = pipeline(model='facebook/bart-large-mnli')
p =  pipe(df_docs['doc'].to_list(), candidate_labels=labels_actions)
for rule_name, result in zip(df_docs.index.values, p):
    print(rule_name, result["labels"][0], result["scores"][0])

start_entry.0 read 0.4533679783344269
start_entry.1 read 0.40628108382225037
start_entry.2 read 0.4566987454891205
start_entry.3 read 0.4434833228588104
start_entry.4 read 0.4864160716533661
start_entry.5 read 0.49792787432670593
$@1 read 0.4286534786224365
sql_statement.0 read 0.6088828444480896
sql_statement.1 read 0.5907757878303528
sql_statement.2 read 0.6675348877906799
opt_end_of_input.0 read 0.4198967218399048
opt_end_of_input.1 execute 0.4449957311153412
simple_statement_or_begin.0 read 0.42838695645332336
simple_statement_or_begin.1 create 0.5969243049621582
simple_statement.0 read 0.5311838388442993
simple_statement.1 read 0.5311838388442993
simple_statement.2 read 0.5311838388442993
simple_statement.3 read 0.6909245848655701
simple_statement.4 read 0.5311838388442993
simple_statement.5 read 0.5311838388442993
simple_statement.6 read 0.6909245848655701
simple_statement.7 read 0.5311838388442993
simple_statement.8 read 0.5311838388442993
simple_statement.9 read 0.5311838388442